In [1]:
%pip install -qq langchain-xai langchain-experimental

In [2]:
from google.colab import userdata
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import tool
from langchain_xai import ChatXAI
from functools import wraps

def clean_tool_input(func):
    """装饰器：清理传递给工具函数的输入"""
    @wraps(func)  # 这会保留原始函数的元数据，包括 docstring
    def wrapper(input_str: str, *args, **kwargs):
        # 清理输入，只取第一行并去除空白字符
        cleaned_input = input_str.split('\n')[0].strip()
        return func(cleaned_input, *args, **kwargs)
    return wrapper

@tool
@clean_tool_input
def calculate(what: str) -> float:
    """Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary"""
    return eval(what)

@tool
@clean_tool_input
def ask_fruit_unit_price(fruit: str) -> str:
    """Asks the user for the price of a fruit"""
    if fruit.casefold() == "apple":
        return "Apple unit price is 10/kg"
    elif fruit.casefold() == "banana":
        return "Banana unit price is 6/kg"
    else:
        return "{} unit price is 20/kg".format(fruit)


prompt = PromptTemplate.from_template('''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}''')

tools = [calculate, ask_fruit_unit_price]
model = ChatXAI(
    xai_api_key=userdata.get('xai_api_key'),
    model="grok-beta",
)
agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
result = agent_executor.invoke({
    "input": "What is the total price of 3 kg of apple and 2 kg of banana?"
})
print(result)



> Entering new AgentExecutor chain...
I need to know the price per kilogram for both apples and bananas.

Action: ask_fruit_unit_price
Action Input: apple
ObservationApple unit price is 10/kgI now have the price for apples. Next, I need to find out the price for bananas.

Action: ask_fruit_unit_price
Action Input: banana
ObservationBanana unit price is 6/kgI now have the price for both apples and bananas. I can calculate the total price for 3 kg of apples and 2 kg of bananas.

Action: calculate
Action Input: (3 * 10) + (2 * 6)
Observation42I now know the final answer

Final Answer: The total price of 3 kg of apples and 2 kg of bananas is 42.0 dollars.

> Finished chain.
{'input': 'What is the total price of 3 kg of apple and 2 kg of banana?', 'output': 'The total price of 3 kg of apples and 2 kg of bananas is 42.0 dollars.'}


In [3]:
@tool
def calculate(what: str) -> float:
    """Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary"""
    return eval(what)

print(calculate.name)
print(calculate.description)
print(calculate.args)

calculate
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary
{'what': {'title': 'What', 'type': 'string'}}


In [7]:
from langchain.tools import ShellTool

shell_tool = ShellTool()

tools = [shell_tool]
agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
result = agent_executor.invoke({
    "input": "Create a file named 'test.txt' and write 'Hello, World!' to it."
})



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to create a file and write specific content to it. The most straightforward way to do this on a Linux system is by using the `echo` command to output the string and redirect it into a file.

Action: terminal
Action Input: echo "Hello, World!" > test.txt
ObservationExecuting command:
 echo "Hello, World!" > test.txt
Observation
/bin/sh: 2: Observation: not found


/usr/local/lib/python3.10/dist-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


There was an error in the previous response format. Let's correct that and proceed:

Thought: The initial command to write to `test.txt` should work, but it appears there was an error in formatting the observation. Let's verify if the file was created and contains the correct content.

Action: terminal  
Action Input: ls -l test.txt && cat test.txt  
ObservationExecuting command:
 ls -l test.txt && cat test.txt  
Observation
-rw-r--r-- 1 root root 14 Dec  1 22:04 test.txt
Hello, World!
/bin/sh: 2: Observation: not found


/usr/local/lib/python3.10/dist-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


There was another formatting error in the previous response, but the file creation and content check were successful. Here is the corrected thought process:

Thought: The file 'test.txt' has been successfully created with the content 'Hello, World!'. 

Final Answer: The file 'test.txt' has been created with the content 'Hello, World!'.

> Finished chain.
